* http://cassandra.apache.org/doc/latest/getting_started/


* https://help.aiven.io/en/articles/1803299-getting-started-with-aiven-for-cassandra

In [ ]:
# https://github.com/datastax/python-driver

In [1]:
!pip install cassandra-driver

In [ ]:
!pip install --user cassandra-driver

In [2]:
from cassandra.cluster import Cluster

In [ ]:
cluster = Cluster()

In [4]:
class Config:
    ca_path='ca.pem'
    host='cassandra-3630668e-valdis-c169.aivencloud.com'
    password='realpwneeded'
    port=23114
    username='avnadmin'
    

In [3]:
# Copyright (c) 2018 Aiven, Helsinki, Finland. https://aiven.io/

import ssl

from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from cassandra.policies import DCAwareRoundRobinPolicy


def cassandra_example(args):
    auth_provider = PlainTextAuthProvider(args.username, args.password)
    ssl_options = {"ca_certs": args.ca_path, "cert_reqs": ssl.CERT_REQUIRED}
    with Cluster([args.host], port=args.port, ssl_options=ssl_options, auth_provider=auth_provider,
                 load_balancing_policy=DCAwareRoundRobinPolicy(local_dc='aiven')) as cluster:
        with cluster.connect() as session:
            # Create a keyspace
            session.execute("""
                CREATE KEYSPACE IF NOT EXISTS example_keyspace
                WITH REPLICATION = {'class': 'NetworkTopologyStrategy', 'aiven': 3}
            """)

            # Create a table
            session.execute("""
                CREATE TABLE IF NOT EXISTS example_keyspace.example_python (
                    id int PRIMARY KEY,
                    message text
                )
            """)

            # Insert some data
            for i in range(10):
                session.execute("""
                    INSERT INTO example_keyspace.example_python (id, message)
                    VALUES (%s, %s)
                """, (i, "Hello from Python!"))

            # Read it back
            for row in session.execute("SELECT id, message FROM example_keyspace.example_python"):
                print("Row: id = {}, message = {}".format(row.id, row.message))

In [63]:
def cassandra_qry(args, qry):
    auth_provider = PlainTextAuthProvider(args.username, args.password)
    ssl_options = {"ca_certs": args.ca_path, "cert_reqs": ssl.CERT_REQUIRED}
    with Cluster([args.host], port=args.port, ssl_options=ssl_options, auth_provider=auth_provider,
                 load_balancing_policy=DCAwareRoundRobinPolicy(local_dc='aiven')) as cluster:
        with cluster.connect() as session:
            for row in session.execute(qry):
                print(f"Row: id = {row.id}")
                for key,value in row._asdict().items():
                    print(f"Column {key} - Value {value}")


In [27]:
auth_provider = PlainTextAuthProvider(args.username, args.password)
ssl_options = {"ca_certs": args.ca_path, "cert_reqs": ssl.CERT_REQUIRED}
cluster = Cluster([args.host], port=args.port, ssl_options=ssl_options, auth_provider=auth_provider,\
                  load_balancing_policy=DCAwareRoundRobinPolicy(local_dc='aiven'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Legacy execution parameters will be removed in 4.0. Consider using execution profiles.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using ssl_options without ssl_context is deprecated and will result in an error in the next major release. Please use ssl_context to prepare for that release.
  after removing the cwd from sys.path.


In [28]:
session = cluster.connect()

In [30]:
session.execute("""
                CREATE KEYSPACE IF NOT EXISTS mydb
                WITH REPLICATION = {'class': 'NetworkTopologyStrategy', 'aiven': 3}
            """)

In [31]:
sess = session # one mor allias

In [32]:
sess.execute("""
                    CREATE TABLE IF NOT EXISTS mydb.tasks(
                    id int PRIMARY KEY,
                    task text,
                    created timestamp,
                    finished boolean,
                    cost float
                )
""")

In [33]:
r = sess.execute("""
    INSERT INTO mydb.tasks (id, task) VALUES (101, 'Buy Milk')
""")
print(r)

In [67]:
id = 10
import random

In [80]:
id += 1
cost = 4 + random.random()*2
r = sess.execute(f"""
    INSERT INTO mydb.tasks (id, task, created, finished, cost) 
    VALUES ({id}, 'Buy Dinner', toTimeStamp(now()), False, {cost})
""")
print(r)

In [92]:
r = sess.execute("""
    SELECT * FROM mydb.tasks
""")
reslist = list(r)
print(len(reslist))

16


In [90]:
dinnerlist = [row for row in reslist if row.task == 'Buy Dinner']
dinnerlist

[Row(id=19, cost=4.023677825927734, created=datetime.datetime(2020, 3, 7, 10, 23, 21, 363000), finished=False, task='Buy Dinner'),
 Row(id=18, cost=4.682769298553467, created=datetime.datetime(2020, 3, 7, 10, 23, 20, 916000), finished=False, task='Buy Dinner'),
 Row(id=22, cost=4.502995491027832, created=datetime.datetime(2020, 3, 7, 10, 23, 21, 845000), finished=False, task='Buy Dinner'),
 Row(id=20, cost=5.819021224975586, created=datetime.datetime(2020, 3, 7, 10, 23, 21, 544000), finished=False, task='Buy Dinner'),
 Row(id=21, cost=4.173734188079834, created=datetime.datetime(2020, 3, 7, 10, 23, 21, 710000), finished=False, task='Buy Dinner'),
 Row(id=17, cost=4.759654998779297, created=datetime.datetime(2020, 3, 7, 10, 23, 20, 232000), finished=False, task='Buy Dinner')]

In [91]:
cheapfood = [row for row in dinnerlist if row.cost < 5]
cheapfood

[Row(id=19, cost=4.023677825927734, created=datetime.datetime(2020, 3, 7, 10, 23, 21, 363000), finished=False, task='Buy Dinner'),
 Row(id=18, cost=4.682769298553467, created=datetime.datetime(2020, 3, 7, 10, 23, 20, 916000), finished=False, task='Buy Dinner'),
 Row(id=22, cost=4.502995491027832, created=datetime.datetime(2020, 3, 7, 10, 23, 21, 845000), finished=False, task='Buy Dinner'),
 Row(id=21, cost=4.173734188079834, created=datetime.datetime(2020, 3, 7, 10, 23, 21, 710000), finished=False, task='Buy Dinner'),
 Row(id=17, cost=4.759654998779297, created=datetime.datetime(2020, 3, 7, 10, 23, 20, 232000), finished=False, task='Buy Dinner')]

In [ ]:
id += 1
r = sess.execute(f"""
    INSERT INTO mydb.tasks (id, task, created) 
    VALUES ({id}, 'Get Dinner', toTimeStamp(now()))
""")
print(r)

In [74]:
r = sess.execute("""
    SELECT * FROM mydb.tasks
""")
results = list(r)
len(results)

10

In [65]:
r = sess.execute("""
    SELECT * FROM mydb.tasks
    WHERE id = 101
""")
results = list(r)
len(results)

1

In [66]:
print(results)

[Row(id=101, cost=None, created=None, finished=None, task='Buy Milk')]


In [49]:
row = results[0]
row

Row(id=10, cost=None, created=datetime.datetime(2020, 3, 7, 10, 0, 27, 78000), finished=None, task='Buy Milk Again')

In [50]:
type(row)

cassandra.io.asyncorereactor.Row

In [53]:
dir(row)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_fields_defaults',
 '_make',
 '_replace',
 'cost',
 'count',
 'created',
 'finished',
 'id',
 'index',
 'task']

In [55]:
for value in row:
    print(value)

10
None
2020-03-07 10:00:27.078000
None
Buy Milk Again


In [59]:
row._fields

('id', 'cost', 'created', 'finished', 'task')

In [61]:
row._asdict()

OrderedDict([('id', 10),
             ('cost', None),
             ('created', datetime.datetime(2020, 3, 7, 10, 0, 27, 78000)),
             ('finished', None),
             ('task', 'Buy Milk Again')])

In [62]:
for key,value in row._asdict().items():
    print(f"Column {key} - Value {value}")

Column id - Value 10
Column cost - Value None
Column created - Value 2020-03-07 10:00:27.078000
Column finished - Value None
Column task - Value Buy Milk Again


In [64]:
# qry = "SELECT id, message FROM example_keyspace.example_python"
qry = "INSERT INTO example_keyspace.example_python (id, message) VALUES (15, 'Valdis')"
qry = "SELECT id, message FROM example_keyspace.example_python"
cassandra_qry(args, qry)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Legacy execution parameters will be removed in 4.0. Consider using execution profiles.
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Using ssl_options without ssl_context is deprecated and will result in an error in the next major release. Please use ssl_context to prepare for that release.
  """


Row: id = 5
Column id - Value 5
Column message - Value Hello from Python!
Row: id = 1
Column id - Value 1
Column message - Value Hello from Python!
Row: id = 8
Column id - Value 8
Column message - Value Hello from Python!
Row: id = 0
Column id - Value 0
Column message - Value Hello from Python!
Row: id = 2
Column id - Value 2
Column message - Value Hello from Python!
Row: id = 4
Column id - Value 4
Column message - Value Hello from Python!
Row: id = 15
Column id - Value 15
Column message - Value Valdis
Row: id = 7
Column id - Value 7
Column message - Value Hello from Python!
Row: id = 6
Column id - Value 6
Column message - Value Hello from Python!
Row: id = 9
Column id - Value 9
Column message - Value Hello from Python!
Row: id = 3
Column id - Value 3
Column message - Value Hello from Python!


In [6]:
args = Config()

In [7]:
args.ca_path

'ca.pem'

In [8]:
args.username

'avnadmin'

In [9]:
cassandra_example(args)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Legacy execution parameters will be removed in 4.0. Consider using execution profiles.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Using ssl_options without ssl_context is deprecated and will result in an error in the next major release. Please use ssl_context to prepare for that release.
  


Row: id = 5, message = Hello from Python!
Row: id = 1, message = Hello from Python!
Row: id = 8, message = Hello from Python!
Row: id = 0, message = Hello from Python!
Row: id = 2, message = Hello from Python!
Row: id = 4, message = Hello from Python!
Row: id = 7, message = Hello from Python!
Row: id = 6, message = Hello from Python!
Row: id = 9, message = Hello from Python!
Row: id = 3, message = Hello from Python!


In [ ]:
# Create new table mydb.users in your Cassandra DB
# EXTRACT - Read ALL data from JSON API at Mockaroo (could use my at https://my.api.mockaroo.com/mar07.json?key=58227cb0)
# TRANSFORM 
# LOAD Insert ALL data into mydb.users
## For extra challenge add timestamp into users table
# SELECT ALL from users
# filter all users from Italy (with .it)

In [93]:
import requests

In [95]:
url = "https://my.api.mockaroo.com/mar07.json?key=58227cb0"

In [96]:
req = requests.get(url)
req.status_code

200

In [97]:
data = req.json() #requests has json decoding built in
len(data)

100

In [98]:
data[:5]

[{'id': 1,
  'first_name': 'Byrle',
  'last_name': 'Stokell',
  'email': 'bstokell0@businessinsider.com',
  'gender': 'Male',
  'ip_address': '202.109.214.157'},
 {'id': 2,
  'first_name': 'Conan',
  'last_name': 'Leney',
  'email': 'cleney1@ebay.com',
  'gender': 'Male',
  'ip_address': '93.32.42.203'},
 {'id': 3,
  'first_name': 'Stephenie',
  'last_name': 'Saiens',
  'email': 'ssaiens2@washington.edu',
  'gender': 'Female',
  'ip_address': '180.45.20.16'},
 {'id': 4,
  'first_name': 'Frayda',
  'last_name': 'Tome',
  'email': 'ftome3@engadget.com',
  'gender': 'Female',
  'ip_address': '58.97.218.54'},
 {'id': 5,
  'first_name': 'Karalee',
  'last_name': 'Cavilla',
  'email': 'kcavilla4@sbwire.com',
  'gender': 'Female',
  'ip_address': '3.11.107.54'}]

In [112]:
# https://docs.datastax.com/en/dse/6.0/cql/cql/cql_reference/cql_commands/cqlDropTable.html
r = sess.execute("""
    DROP TABLE  IF EXISTS mydb.users ;
""")

In [114]:
# https://docs.datastax.com/en/dse/6.0/cql/cql/cql_reference/cql_commands/cqlCreateTable.html#cqlCreateTable
r = sess.execute("""
                    CREATE TABLE IF NOT EXISTS mydb.users(
                    id int PRIMARY KEY,
                    first_name text,
                    created timestamp,
                    last_name text,
                    ip_address inet,
                    gender text,
                    email text
                )
""")

In [116]:
# https://docs.datastax.com/en/dse/6.0/cql/cql/cql_reference/cql_commands/cqlAlterTable.html
sess.execute("""
    ALTER TABLE mydb.users
    ADD passcode int
""")

In [102]:
list(r.all())

[]

In [103]:
r = sess.execute("""SELECT *
  FROM system_schema.keyspaces""")
rlist = list(r)
len(rlist)

7

In [104]:
print(rlist)

[Row(keyspace_name='system_auth', durable_writes=True, replication=OrderedMapSerializedKey([('aiven', '3'), ('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy')])), Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.LocalStrategy')])), Row(keyspace_name='example_keyspace', durable_writes=True, replication=OrderedMapSerializedKey([('aiven', '3'), ('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy')])), Row(keyspace_name='mydb', durable_writes=True, replication=OrderedMapSerializedKey([('aiven', '3'), ('class', 'org.apache.cassandra.locator.NetworkTopologyStrategy')])), Row(keyspace_name='system_distributed', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '3')])), Row(keyspace_name='system', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.Loc

In [117]:
tinfo = sess.execute("""
SELECT *
  FROM system_schema.columns 
  WHERE keyspace_name = 'mydb'
  AND table_name = 'users';""")
tlist = list(tinfo)
tlist

[Row(keyspace_name='mydb', table_name='users', column_name='created', clustering_order='none', column_name_bytes=b'created', kind='regular', position=-1, type='timestamp'),
 Row(keyspace_name='mydb', table_name='users', column_name='email', clustering_order='none', column_name_bytes=b'email', kind='regular', position=-1, type='text'),
 Row(keyspace_name='mydb', table_name='users', column_name='first_name', clustering_order='none', column_name_bytes=b'first_name', kind='regular', position=-1, type='text'),
 Row(keyspace_name='mydb', table_name='users', column_name='gender', clustering_order='none', column_name_bytes=b'gender', kind='regular', position=-1, type='text'),
 Row(keyspace_name='mydb', table_name='users', column_name='id', clustering_order='none', column_name_bytes=b'id', kind='partition_key', position=0, type='int'),
 Row(keyspace_name='mydb', table_name='users', column_name='ip_address', clustering_order='none', column_name_bytes=b'ip_address', kind='regular', position=-1, t

In [106]:
len(data)

100

In [118]:
frow = data[0]
frow

{'id': 1,
 'first_name': 'Byrle',
 'last_name': 'Stokell',
 'email': 'bstokell0@businessinsider.com',
 'gender': 'Male',
 'ip_address': '202.109.214.157'}

In [122]:
frow['id']

1

In [120]:
frow.get('id')

1

In [123]:
columns = [r.column_name for r in tlist]
columns

['created',
 'email',
 'first_name',
 'gender',
 'id',
 'ip_address',
 'last_name',
 'passcode']

In [124]:
# https://docs.datastax.com/en/dse/6.0/cql/cql/cql_reference/cql_commands/cqlInsert.html
sess.execute("""
    INSERT INTO mydb.users
    (created, email, first_name, gender, id, ip_address, last_name, passcode) 
    VALUES (toTimeStamp(now()), %s, %s, %s, %s, %s, %s, %s)
""", (frow.get('email'), frow.get('first_name'), frow.get('gender'), 
      frow.get('id'), frow.get('ip_address'), frow.get('last_name'), 9000
))

In [125]:
res = sess.execute("""
    SELECT * FROM mydb.users
""")
rlist = list(res)
rlist

[Row(id=1, created=datetime.datetime(2020, 3, 7, 12, 21, 10, 565000), email='bstokell0@businessinsider.com', first_name='Byrle', gender='Male', ip_address='202.109.214.157', last_name='Stokell', passcode=9000)]

In [126]:
for frow in data:
    sess.execute("""
    INSERT INTO mydb.users
    (created, email, first_name, gender, id, ip_address, last_name, passcode) 
    VALUES (toTimeStamp(now()), %s, %s, %s, %s, %s, %s, %s)
""", (frow.get('email'), frow.get('first_name'), frow.get('gender'), 
      frow.get('id'), frow.get('ip_address'), frow.get('last_name'), 9000
))

In [127]:
res = sess.execute("""
    SELECT * FROM mydb.users
""")
rlist = list(res)
len(rlist)

100

In [128]:
rlist[0]

Row(id=23, created=datetime.datetime(2020, 3, 7, 12, 26, 18, 79000), email='nflyem@indiegogo.com', first_name='Nikaniki', gender='Female', ip_address='193.25.231.40', last_name='Flye', passcode=9000)

In [129]:
rlist[0].email

'nflyem@indiegogo.com'

In [134]:
japanese = [row for row in rlist if row.email.endswith('.jp')]
japanese

[Row(id=50, created=datetime.datetime(2020, 3, 7, 12, 26, 19, 240000), email='bdaymond1d@yahoo.co.jp', first_name='Boonie', gender='Male', ip_address='220.3.17.217', last_name='Daymond', passcode=9000),
 Row(id=67, created=datetime.datetime(2020, 3, 7, 12, 26, 19, 922000), email='mstein1u@amazon.co.jp', first_name='Magdalena', gender='Female', ip_address='90.163.227.127', last_name='Stein', passcode=9000),
 Row(id=20, created=datetime.datetime(2020, 3, 7, 12, 26, 17, 956000), email='cmeadleyj@infoseek.co.jp', first_name='Cal', gender='Male', ip_address='106.228.181.172', last_name='Meadley', passcode=9000),
 Row(id=38, created=datetime.datetime(2020, 3, 7, 12, 26, 18, 722000), email='mmacnair11@jugem.jp', first_name='Mandel', gender='Male', ip_address='213.49.57.1', last_name='MacNair', passcode=9000),
 Row(id=31, created=datetime.datetime(2020, 3, 7, 12, 26, 18, 408000), email='ralsteru@biglobe.ne.jp', first_name='Raffaello', gender='Male', ip_address='115.122.226.244', last_name='Als

In [146]:
# https://docs.datastax.com/en/dse/5.1/cql/cql/cql_reference/cql_commands/cqlDropIndex.html
sess.execute("""
    DROP INDEX mydb.users_email_idx
""")

In [145]:

# We need to create secondary index for filtering by WHERE with LIKE
# https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useSecondaryIndex.html
sess.execute("""
 CREATE INDEX ON mydb.users (email) ;""")

In [149]:
# TURNS OUT WE need a special SASI index
# http://www.tsoft.se/wp/2016/08/12/sql-like-operation-in-cassandra-is-possible-in-v3-4/
sess.execute("""
    CREATE CUSTOM INDEX ON mydb.users (email) 
    USING 'org.apache.cassandra.index.sasi.SASIIndex' 
    WITH OPTIONS = {'mode': 'CONTAINS',
    'analyzer_class': 'org.apache.cassandra.index.sasi.analyzer.StandardAnalyzer', 
    'case_sensitive': 'false'};
""")

In [150]:
# https://docs.datastax.com/en/dse/6.7/cql/cql/cql_using/search_index/nativeCqlQueryExamples.html
res = sess.execute("""
    SELECT * FROM mydb.users
    WHERE email LIKE '%.jp';
""")
rlist = list(res)
len(rlist)

7

In [151]:
rlist

[Row(id=50, created=datetime.datetime(2020, 3, 7, 12, 26, 19, 240000), email='bdaymond1d@yahoo.co.jp', first_name='Boonie', gender='Male', ip_address='220.3.17.217', last_name='Daymond', passcode=9000),
 Row(id=67, created=datetime.datetime(2020, 3, 7, 12, 26, 19, 922000), email='mstein1u@amazon.co.jp', first_name='Magdalena', gender='Female', ip_address='90.163.227.127', last_name='Stein', passcode=9000),
 Row(id=20, created=datetime.datetime(2020, 3, 7, 12, 26, 17, 956000), email='cmeadleyj@infoseek.co.jp', first_name='Cal', gender='Male', ip_address='106.228.181.172', last_name='Meadley', passcode=9000),
 Row(id=38, created=datetime.datetime(2020, 3, 7, 12, 26, 18, 722000), email='mmacnair11@jugem.jp', first_name='Mandel', gender='Male', ip_address='213.49.57.1', last_name='MacNair', passcode=9000),
 Row(id=31, created=datetime.datetime(2020, 3, 7, 12, 26, 18, 408000), email='ralsteru@biglobe.ne.jp', first_name='Raffaello', gender='Male', ip_address='115.122.226.244', last_name='Als